**Chapter 8: Advanced Docker Techniques**

Basic Dockerfiles create functional containers; production environments demand optimized, secure, and efficient artifacts. This chapter explores advanced patterns that reduce image size, accelerate build times, harden security postures, and ensure reliable runtime behavior. These techniques separate development experiments from enterprise-grade containerization ready for Kubernetes orchestration.

---

### 8.1 Multi-Stage Builds

Multi-stage builds allow you to use multiple `FROM` statements in a single Dockerfile. Each stage can use a different base, and you can selectively copy artifacts between stages, leaving behind build tools, source code, and intermediate files in the final image.

#### The Builder Pattern

Separate compilation/build tools from runtime dependencies:

```dockerfile
# syntax=docker/dockerfile:1
# Stage 1: Build environment with full toolchain
FROM golang:1.21 AS builder
WORKDIR /build

# Install build dependencies
RUN apt-get update && apt-get install -y git build-essential

# Cache dependency downloads
COPY go.mod go.sum ./
RUN go mod download

# Build the application
COPY . .
RUN CGO_ENABLED=0 GOOS=linux go build -a -installsuffix cgo -o app ./cmd/server

# Stage 2: Minimal runtime
FROM gcr.io/distroless/static:nonroot
WORKDIR /app

# Copy only the compiled binary
COPY --from=builder /build/app /app/

# Copy CA certificates for HTTPS
COPY --from=builder /etc/ssl/certs/ca-certificates.crt /etc/ssl/certs/

USER nonroot:nonroot
EXPOSE 8080
ENTRYPOINT ["/app/app"]
```

**Size Impact:**
- Single stage with Go toolchain: ~1.2GB
- Multi-stage with distroless: ~25MB (98% reduction)

#### Named Stages and Parallel Builds

Name stages for clarity and potential parallelization:

```dockerfile
# syntax=docker/dockerfile:1
FROM node:20-alpine AS dependencies
WORKDIR /app
COPY package*.json ./
RUN npm ci --only=production

FROM node:20-alpine AS development
WORKDIR /app
COPY --from=dependencies /app/node_modules ./node_modules
COPY . .
CMD ["npm", "run", "dev"]

FROM node:20-alpine AS production
WORKDIR /app
COPY --from=dependencies /app/node_modules ./node_modules
COPY . .
RUN npm run build
RUN npm prune --production
USER node
CMD ["node", "dist/index.js"]
```

**Build specific targets:**
```bash
# Build only development stage
docker build --target development -t myapp:dev .

# Build only production
docker build --target production -t myapp:prod .
```

#### Cross-Stage Dependencies

Access files from any previous stage, not just immediately preceding:

```dockerfile
FROM node:20-alpine AS builder
WORKDIR /app
COPY . .
RUN npm ci && npm run build

FROM nginx:alpine AS assets
COPY --from=builder /app/dist /usr/share/nginx/html

FROM builder AS test-runner
RUN npm test

FROM assets AS production
# Only assets make it here, test artifacts excluded
```

#### Security Isolation

Use multi-stage builds to handle secrets without leaving them in layers:

```dockerfile
# Stage with SSH key for private repo access
FROM alpine AS git-clone
RUN apk add --no-cache git openssh-client
COPY id_rsa /root/.ssh/id_rsa
RUN chmod 600 /root/.ssh/id_rsa
RUN git clone git@github.com:private/repo.git /src

# Final stage - no SSH key
FROM nginx:alpine
COPY --from=git-clone /src/static /usr/share/nginx/html
```

**Key Takeaway:** Multi-stage builds are the **primary optimization technique** for production images. They eliminate build tools, source code, and secrets from final artifacts, reducing attack surface and image size by orders of magnitude.

---

### 8.2 Build Cache Optimization

Docker's layer cache dramatically accelerates rebuilds, but naive Dockerfile ordering invalidates caches unnecessarily. Strategic ordering ensures expensive operations (dependency installation) reuse cache while cheap operations (code changes) invalidate only final layers.

#### Cache Invalidation Rules

A layer is rebuilt if:
- The Dockerfile instruction changes
- The files referenced by `COPY` or `ADD` change (checksum comparison)
- Any previous layer was rebuilt

**The Golden Rule:** Order from least-changing to most-changing.

```dockerfile
# BAD - Changes to source invalidate dependency layer
COPY . /app
RUN npm install

# GOOD - Dependencies cached unless package.json changes
COPY package*.json /app/
RUN npm install
COPY . /app
```

#### Dependency Cache Mounts (BuildKit)

Modern Docker with BuildKit supports cache mounts that persist between builds without creating layers:

```dockerfile
# syntax=docker/dockerfile:1
FROM python:3.11-slim
WORKDIR /app

# Mount cache directory for pip
RUN --mount=type=cache,target=/root/.cache/pip \
    pip install -r requirements.txt
```

**Benefits:**
- Cache persists across `docker build` invocations
- Not included in final image layers
- Shared between parallel builds

#### Apt/Apk Cache Persistence

Speed up package installation in CI:

```dockerfile
# syntax=docker/dockerfile:1
FROM ubuntu:22.04
RUN --mount=type=cache,target=/var/cache/apt,sharing=locked \
    --mount=type=cache,target=/var/lib/apt,sharing=locked \
    apt-get update && apt-get install -y \
    build-essential \
    libpq-dev
```

#### Cache Invalidation Strategies

**Version Pinning with Cache Busting:**
```dockerfile
ARG DEPENDENCIES_VERSION=1.0
COPY requirements.txt /tmp/requirements.txt
RUN --mount=type=cache,target=/root/.cache/pip,id=pip-${DEPENDENCIES_VERSION} \
    pip install -r /tmp/requirements.txt
```

Force cache invalidation by changing the ARG:
```bash
docker build --build-arg DEPENDENCIES_VERSION=1.1 .
```

#### Layer Size Analysis

Identify large layers slowing builds:

```bash
# Show layer sizes
docker history myimage:latest

# Detailed analysis with dive tool
dive myimage:latest
```

**Optimization targets:**
- Keep dependency layers under 500MB
- Minimize RUN commands that produce large temporary files
- Clean up package managers in same layer as install

**Key Takeaway:** Build cache optimization reduces CI/CD pipeline times from minutes to seconds. Structure Dockerfiles to maximize cache hits on expensive operations, and use BuildKit cache mounts for package managers.

---

### 8.3 BuildKit Features

BuildKit is Docker's modern builder with advanced features for security, performance, and flexibility. Enable it with `export DOCKER_BUILDKIT=1` or configure in daemon.json.

#### Secret Mounts

Pass secrets to build without committing to image history:

```dockerfile
# syntax=docker/dockerfile:1
FROM node:20-alpine
WORKDIR /app
COPY package*.json ./

# Mount npm token as secret
RUN --mount=type=secret,id=npmrc,target=/root/.npmrc \
    npm ci

COPY . .
RUN npm run build
```

**Build command:**
```bash
# Create temporary secret file
echo "//registry.npmjs.org/:_authToken=${NPM_TOKEN}" > npmrc

# Build with secret (not stored in layers)
DOCKER_BUILDKIT=1 docker build \
  --secret id=npmrc,src=npmrc \
  -t myapp:latest .

# Remove secret file
rm npmrc
```

#### SSH Forwarding

Clone private repositories during build:

```dockerfile
# syntax=docker/dockerfile:1
FROM alpine AS builder
RUN apk add --no-cache git openssh-client

# Mount SSH agent socket
RUN --mount=type=ssh git clone git@github.com:private/repo.git /src

FROM scratch
COPY --from=builder /src/app /app
```

**Build command:**
```bash
eval $(ssh-agent -s)
ssh-add ~/.ssh/id_rsa

docker buildx build --ssh default=${SSH_AUTH_SOCK} -t myapp:latest .
```

#### Bind Mounts During Build

Access host files without COPY (useful for large files):

```dockerfile
# syntax=docker/dockerfile:1
FROM python:3.11
WORKDIR /app

# Mount source code read-only instead of COPY
RUN --mount=type=bind,source=.,target=/src,ro \
    pip install /src

# Or mount build cache from host
RUN --mount=type=bind,source=/host/cache,target=/cache \
    cp -r /cache/dependencies /usr/local/lib
```

#### Parallel Stage Execution

BuildKit builds independent stages concurrently:

```dockerfile
# syntax=docker/dockerfile:1
FROM node:20-alpine AS lint
WORKDIR /app
COPY . .
RUN npm run lint

FROM node:20-alpine AS test
WORKDIR /app
COPY . .
RUN npm test

FROM node:20-alpine AS build
WORKDIR /app
COPY . .
RUN npm run build

# Final stage waits for all three
FROM nginx:alpine AS production
COPY --from=build /app/dist /usr/share/nginx/html
```

**All three stages (lint, test, build) execute simultaneously** on multi-core systems.

**Key Takeaway:** BuildKit transforms Docker from a simple layer builder into a sophisticated build system. Secret mounts eliminate credential leakage, SSH forwarding enables private repository access, and parallel execution maximizes hardware utilization.

---

### 8.4 Custom Networks

While basic container networking suffices for simple cases, production applications require custom network topologies, static IPs, and cross-container communication patterns.

#### User-Defined Bridge Networks

Create isolated networks with custom DNS and IPAM:

```bash
# Create custom network
docker network create \
  --driver bridge \
  --subnet=172.28.0.0/16 \
  --ip-range=172.28.5.0/24 \
  --gateway=172.28.5.254 \
  --opt com.docker.network.bridge.name=br-custom \
  custom-network

# Run container in specific network with static IP
docker run -d --name db \
  --network custom-network \
  --ip 172.28.5.10 \
  postgres:15
```

#### Container DNS and Aliases

Custom DNS entries for service discovery:

```bash
# Multiple network aliases
docker run -d --name api \
  --network custom-network \
  --network-alias api \
  --network-alias backend \
  --network-alias api.internal \
  myapp:latest
```

#### Macvlan Networks (Production Parity)

Attach containers directly to physical network (gets real IP):

```bash
# Create macvlan network
docker network create -d macvlan \
  --subnet=192.168.1.0/24 \
  --gateway=192.168.1.1 \
  -o parent=eth0 \
  macvlan-net

# Container gets IP like 192.168.1.100 (routable on LAN)
docker run -d --network macvlan-net myapp
```

**Use case:** Legacy applications expecting real MAC addresses, or when containers need to appear as physical hosts on the network.

#### IPv6 Configuration

Enable IPv6 for future-proofing:

```bash
docker network create \
  --ipv6 \
  --subnet=2001:db8:1::/64 \
  --gateway=2001:db8:1::1 \
  ipv6-network
```

**Key Takeaway:** Custom networks provide **isolation, DNS control, and IP management** essential for multi-tier applications. They simulate Kubernetes network policies and service discovery in local environments.

---

### 8.5 Volume Management

Beyond basic persistence, Docker supports advanced volume drivers, volume pre-population, and performance optimization.

#### Volume Drivers

Use external storage systems:

```bash
# NFS volume for shared storage across hosts
docker volume create --driver local \
  -o type=nfs \
  -o o=addr=192.168.1.100,rw \
  -o device=:/path/to/export \
  nfs-volume

# Use in container
docker run -v nfs-volume:/data myapp
```

#### Volume Initialization Patterns

Populate volumes on first run:

```dockerfile
FROM alpine AS builder
RUN mkdir /data && echo "initial config" > /data/config.conf

FROM alpine
VOLUME /data
COPY --from=builder /data/config.conf /data/
CMD ["cat", "/data/config.conf"]
```

**Note:** `VOLUME` in Dockerfile creates an anonymous volume if not mapped, but content in image at that path is copied to volume on first run.

#### tmpfs Mounts

In-memory storage for sensitive or temporary data:

```bash
docker run -d \
  --tmpfs /tmp:noexec,nosuid,size=100m \
  --tmpfs /run/secrets:ro,size=10m \
  myapp
```

**Benefits:**
- Data never touches disk (security)
- Extremely fast I/O
- Automatically cleared on container stop

#### Volume Backup Strategies

**Backup:**
```bash
# Create backup container
docker run --rm \
  -v myapp-data:/source:ro \
  -v $(pwd):/backup \
  alpine tar czf /backup/backup.tar.gz -C /source .
```

**Restore:**
```bash
docker run --rm \
  -v myapp-data:/target \
  -v $(pwd):/backup \
  alpine tar xzf /backup/backup.tar.gz -C /target
```

**Key Takeaway:** Advanced volume management supports **stateful applications, shared storage, and security requirements**. Understand volume drivers for production storage integration and tmpfs for sensitive temporary data.

---

### 8.6 Container Resource Limits

Unconstrained containers can exhaust host resources. Production deployments require explicit limits.

#### Memory Constraints

```bash
# Hard limit (OOM kill if exceeded)
docker run -m 512m --memory-swap 512m myapp

# Soft limit (reservation for scheduling)
docker run -m 1g --memory-reservation 512m myapp

# Disable OOM killer (dangerous, use carefully)
docker run -m 512m --oom-kill-disable myapp
```

**In Dockerfile:**
```dockerfile
FROM node:20-alpine
# Runtime enforcement
CMD ["node", "--max-old-space-size=400", "index.js"]
```

#### CPU Constraints

```bash
# Relative weight (shares)
docker run -c 512 myapp  # Default is 1024

# Hard limit (1.5 cores)
docker run --cpus="1.5" myapp

# CPU pinning (specific cores)
docker run --cpuset-cpus="0,1" myapp
docker run --cpuset-cpus="0-3" myapp
```

#### PID Limits

Prevent fork bombs:

```bash
docker run --pids-limit 100 myapp
```

#### ulimits

Control file descriptors and processes:

```bash
docker run \
  --ulimit nofile=65536:65536 \
  --ulimit nproc=4096:4096 \
  myapp
```

#### cgroup v2 Considerations

Modern Linux uses cgroup v2 with different resource controls:

```bash
# Memory pressure control (cgroup v2)
docker run \
  --memory-high 800m \
  --memory-max 1g \
  myapp
```

**Key Takeaway:** Resource limits prevent **noisy neighbor problems** and ensure predictable performance. Always set memory limits in production; without them, a memory leak consumes all host RAM, crashing the system.

---

### 8.7 Signal Handling

Proper signal handling ensures graceful shutdowns—critical for zero-downtime deployments in Kubernetes.

#### The PID 1 Problem

Docker containers run the ENTRYPOINT/CMD as PID 1. PID 1 has special signal handling in Linux: it does not have default signal handlers, so SIGTERM is ignored unless the process explicitly handles it.

**Problem:**
```dockerfile
FROM ubuntu
CMD sleep 1000
```
```bash
docker run -d --name test myimage
docker stop test  # Takes 10 seconds (SIGTERM ignored, then SIGKILL)
```

#### Solutions

**1. Use Exec Form (JSON array):**
```dockerfile
CMD ["sleep", "1000"]  # sleep is PID 1, handles signals
```

**2. Use tini (init system):**
```dockerfile
FROM node:20-alpine
RUN apk add --no-cache tini
ENTRYPOINT ["/sbin/tini", "--"]
CMD ["node", "server.js"]
```

**3. Use --init flag:**
```bash
docker run --init myapp  # Injects tini automatically
```

**4. Signal handling in application code:**

Node.js example:
```javascript
process.on('SIGTERM', () => {
  console.log('SIGTERM received, shutting down gracefully');
  server.close(() => {
    process.exit(0);
  });
});
```

Go example:
```go
sigChan := make(chan os.Signal, 1)
signal.Notify(sigChan, syscall.SIGTERM, syscall.SIGINT)

go func() {
  <-sigChan
  log.Println("Shutting down gracefully...")
  srv.Shutdown(context.Background())
}()
```

#### Graceful Shutdown Timeout

Docker waits 10 seconds (configurable) before SIGKILL:

```bash
# Increase timeout for slow cleanup
docker stop -t 30 mycontainer
docker kill -s SIGTERM mycontainer  # Custom signal
```

**In Compose:**
```yaml
services:
  api:
    stop_signal: SIGTERM
    stop_grace_period: 30s
```

**Key Takeaway:** Improper signal handling causes **abrupt terminations**, leading to corrupted data, dropped connections, and failed health checks. Always ensure PID 1 properly handles SIGTERM for graceful shutdowns.

---

### 8.8 Security Hardening

Production containers require defense in depth: minimal users, read-only filesystems, dropped capabilities, and security profiles.

#### Non-Root Execution

```dockerfile
FROM node:20-alpine

# Create user with fixed UID/GID (required for Kubernetes security contexts)
RUN addgroup -g 1001 -S nodejs && \
    adduser -S nodejs -u 1001 -G nodejs

WORKDIR /app
COPY --chown=nodejs:nodejs . .

USER nodejs:nodejs
```

**Verification:**
```bash
docker run --rm myimage id
# uid=1001(nodejs) gid=1001(nodejs) groups=1001(nodejs)
```

#### Read-Only Root Filesystem

Prevent runtime modifications:

```bash
docker run --read-only myimage
```

**Handling necessary writes:**
```dockerfile
# Create writable volumes for temp/cache
VOLUME ["/tmp", "/var/cache", "/app/logs"]
```

Or in Compose:
```yaml
services:
  app:
    read_only: true
    tmpfs:
      - /tmp
      - /run
    volumes:
      - app-logs:/app/logs
```

#### Dropping Capabilities

Remove unnecessary Linux capabilities:

```bash
docker run --cap-drop=ALL --cap-add=NET_BIND_SERVICE myimage
```

**Common capabilities to add back:**
- `NET_BIND_SERVICE`: Bind to ports <1024
- `SETGID`, `SETUID`: Change user/group (if needed)
- `CHOWN`: Change file ownership

#### Security Profiles

**Seccomp (System Call Filtering):**
```bash
# Use default profile (recommended)
docker run --security-opt seccomp=default myimage

# Custom profile
docker run --security-opt seccomp=/path/to/seccomp.json myimage
```

**AppArmor:**
```bash
docker run --security-opt apparmor=docker-default myimage
```

#### No New Privileges

Prevent privilege escalation (sudo, setuid binaries):

```bash
docker run --security-opt no-new-privileges:true myimage
```

#### Distroless and Scratch

Minimal attack surface:

```dockerfile
FROM gcr.io/distroless/static:nonroot
# No shell, no package manager, no unnecessary binaries
COPY --chown=nonroot:nonroot app /app
USER nonroot
ENTRYPOINT ["/app"]
```

**Scanning for vulnerabilities:**
```bash
docker run --rm -v /var/run/docker.sock:/var/run/docker.sock \
  aquasec/trivy image myimage:latest
```

**Key Takeaway:** Security hardening transforms containers from **convenient packaging** into **secure isolation boundaries**. Run as non-root, use read-only filesystems, drop capabilities, and prefer Distroless bases for production workloads.

---

### Chapter Summary and Preview

In this chapter, you mastered production-grade containerization techniques. You implemented **multi-stage builds** to separate build tools from runtime artifacts, achieving 90%+ size reductions. You optimized **build caching** through strategic layer ordering and BuildKit cache mounts, dramatically accelerating CI pipelines. You leveraged **BuildKit secrets and SSH forwarding** to handle credentials securely without image contamination. You configured **custom networks and volumes** for complex topologies, enforced **resource limits** to prevent system exhaustion, ensured proper **signal handling** for graceful shutdowns, and applied **security hardening** including non-root users, read-only filesystems, and dropped capabilities.

These techniques produce containers that are fast to build, small to transfer, secure to run, and reliable to terminate—the exact standards required for Kubernetes orchestration. Your images are now ready for the distributed, dynamic environment of container clusters.

In **Chapter 9: Docker Image Optimization**, we refine these techniques further with specific focus on size analysis, layer inspection, and performance benchmarking. You will learn to use tools like `dive` to visualize layer efficiency, implement `.dockerignore` patterns to minimize build context, and benchmark startup performance. This optimization chapter completes our Docker mastery before transitioning to **Part III: Kubernetes Fundamentals**, where these optimized images become the deployable units orchestrated across clusters.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='7. docker_compose_for_local_development.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='9. docker_image_optimization.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
